# Step 3: Build joint model posteriors #

### p( gamma_lens, delta_phi, c $\sqrt{J}$, b_ani | d_img, nu_int) ###

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
import h5py
from scipy.stats import truncnorm
import sys
sys.path.insert(0, '/Users/smericks/Desktop/StrongLensing/darkenergy-from-LAGN/')
import Utils.make_data_vectors as mdv

In [ ]:
# set the anistropy "modeling prior" here
BETA_ANI_MU = 0.
BETA_ANI_SIGMA = 0.4 

In [ ]:
# load in NPE models

npe_models = {
    'gold':{},
    'silver':{}
}

for quality in ['gold','silver']:
    # load in NPE models
    images_path = 'DataVectors/'+quality+'/image_models.h5'
    h5 = h5py.File(images_path, 'r')
    
    npe_models[quality]['mu_npe'] = h5['mu_npe'][:]
    npe_models[quality]['cov_npe'] = h5['cov_npe'][:]
    h5.close()

    # load in ground truth
    df_path = 'DataVectors/'+quality+'/truth_metadata.csv'
    truth_df = pd.read_csv(df_path)
    npe_models[quality]['truth_df'] = truth_df

Compute fermat potential samples and write to .h5 files that store the lens model posterior samples

In [ ]:
# generate fermat potential + lens param samples
#npe_models['gold']['quad_posteriors']['fpd_samps']
#npe_models['gold']['quad_posteriors']['lens_param_samps']
#npe_models['gold']['quad_posteriors']['beta_ani_samps']

gold_metadata_df = npe_models['gold']['truth_df']
silver_metadata_df = npe_models['silver']['truth_df']

dbls_idxs = np.where(gold_metadata_df.loc[:,'point_source_parameters_num_images'].to_numpy() == 2.)[0]
quads_idxs = np.where(gold_metadata_df.loc[:,'point_source_parameters_num_images'].to_numpy() == 4.)[0]

# NOTE: hardcodings in this function for doubles and quads idxs
NUM_FPD_SAMPS = int(500) 
def fpd_samps_helper(metadata_df,npe_mu,npe_cov,num_images):

    if num_images == 2:
        return mdv.fpd_gamma_samples(
            x_im=metadata_df.loc[dbls_idxs,
                ['point_source_parameters_x_image_0',
                'point_source_parameters_x_image_1']].to_numpy(),
            y_im=metadata_df.loc[dbls_idxs,
                ['point_source_parameters_y_image_0',
                'point_source_parameters_y_image_1']].to_numpy(),
            npe_mu=npe_mu[dbls_idxs],npe_cov=npe_cov[dbls_idxs],
            num_fpd_samps=NUM_FPD_SAMPS)
    
    elif num_images == 4:
        return mdv.fpd_gamma_samples(
            x_im=metadata_df.loc[quads_idxs,
                ['point_source_parameters_x_image_0',
                'point_source_parameters_x_image_1',
                'point_source_parameters_x_image_2',
                'point_source_parameters_x_image_3']].to_numpy(),
            y_im=metadata_df.loc[quads_idxs,
                ['point_source_parameters_y_image_0',
                'point_source_parameters_y_image_1',
                'point_source_parameters_y_image_2',
                'point_source_parameters_y_image_3']].to_numpy(),
            npe_mu=npe_mu[quads_idxs],npe_cov=npe_cov[quads_idxs],
            num_fpd_samps=NUM_FPD_SAMPS)
    
    else:
        raise ValueError("Num images not supported")

# gold quads
fpd_samps_gold_quads, lens_param_samps_gold_quads = fpd_samps_helper(
    gold_metadata_df,npe_models['gold']['mu_npe'],
    npe_models['gold']['cov_npe'],num_images=int(4))
# gold doubles
fpd_samps_gold_dbls, lens_param_samps_gold_dbls = fpd_samps_helper(
    gold_metadata_df,npe_models['gold']['mu_npe'],npe_models['gold']['cov_npe'],num_images=2.)
# silver quads
fpd_samps_silver_quads, lens_param_samps_silver_quads = fpd_samps_helper(
    silver_metadata_df,npe_models['silver']['mu_npe'],npe_models['silver']['cov_npe'],num_images=4.)
# silver doubles
fpd_samps_silver_dbls, lens_param_samps_silver_dbls = fpd_samps_helper(
    silver_metadata_df,npe_models['silver']['mu_npe'],npe_models['silver']['cov_npe'],num_images=2.)

In [ ]:
# write .h5 and store some beta_ani samps as well...

def write_h5(h5_name,fpd_samps,lens_param_samps,catalog_idxs):

    # FIRST: generate some beta_ani samps!
    # truncate at beta_ani = -1...
    beta_ani_samps = truncnorm.rvs(
        (-1 - BETA_ANI_MU)/BETA_ANI_SIGMA, # lower truncation at -1
        (1 - BETA_ANI_MU)/BETA_ANI_SIGMA, # upper truncation at 1
        loc=BETA_ANI_MU,scale=BETA_ANI_SIGMA,
        size=(fpd_samps_gold_quads.shape[0],fpd_samps_gold_quads.shape[1]))

    # now, write the .h5 file and store all the samps...
    h5f = h5py.File(h5_name, 'w')
    h5f.create_dataset('catalog_idxs', data=catalog_idxs)
    h5f.create_dataset('fpd_samps', data=fpd_samps)
    h5f.create_dataset('lens_param_samps', data=lens_param_samps)
    h5f.create_dataset('beta_ani_samps', data=beta_ani_samps)

    # TODO: initialize empty kinematics??
    h5f.create_dataset('c_sqrtJ_samps', data=-1*np.ones(
        (fpd_samps_gold_quads.shape[0],fpd_samps_gold_quads.shape[1],1)))
    h5f.close()

# gold quads
write_h5('DataVectors/gold/quad_posteriors.h5',
    fpd_samps_gold_quads,lens_param_samps_gold_quads,
    catalog_idxs=gold_metadata_df.loc[quads_idxs,'catalog_idx'].to_numpy())
# gold doubles
write_h5('DataVectors/gold/dbl_posteriors.h5',
    fpd_samps_gold_dbls,lens_param_samps_gold_dbls,
    catalog_idxs=gold_metadata_df.loc[dbls_idxs,'catalog_idx'].to_numpy())
# silver quads
write_h5('DataVectors/silver/quad_posteriors.h5',
    fpd_samps_silver_quads,lens_param_samps_silver_quads,
    catalog_idxs=silver_metadata_df.loc[quads_idxs,'catalog_idx'].to_numpy())
# silver doubles
write_h5('DataVectors/silver/dbl_posteriors.h5',
    fpd_samps_silver_dbls,lens_param_samps_silver_dbls,
    catalog_idxs=silver_metadata_df.loc[dbls_idxs,'catalog_idx'].to_numpy())

Now, we have to add in the kinematic model values that are consistent with the lens model samples. This is computationally expensive, so we develop code here, and then send to a cluster (takes ~30mins per lens for 500 samples)

In [ ]:
# test how to write in kinematics into the .h5

from filelock import FileLock
import Modeling.Kinematics.galkin_utils as galkin_utils

h5_posteriors_file = 'DataVectors/gold/quad_posteriors.h5'

# read in fpd samps, lens_param_samps, beta_ani_samps
with FileLock(h5_posteriors_file + ".lock"):

    h5 = h5py.File(h5_posteriors_file, 'r')
    fpd_samps = h5['fpd_samps'][:]
    lens_param_samps = h5['lens_param_samps'][:]
    beta_ani_samps = h5['beta_ani_samps'][:]
    catalog_idxs = h5['catalog_idxs'][:]
    h5.close()

# read in truth metadata
truth_df = pd.read_csv('DataVectors/gold/truth_metadata.csv')

# compute kinematics

n_fpd_samps = np.shape(fpd_samps)[1]
c_sqrtJ_samps = np.empty((n_fpd_samps,1))
lens_idx = 0
catalog_idx = catalog_idxs[lens_idx]
R_sersic_truth = truth_df.loc[truth_df['catalog_idx']==catalog_idx,'lens_light_parameters_R_sersic'].item()
n_sersic_truth = truth_df.loc[truth_df['catalog_idx']==catalog_idx,'lens_light_parameters_n_sersic'].item()

# NOTE: only computing 10 samples (just to test things are working...)
for fp_idx in range(0,10):

    csqrtJ = galkin_utils.ground_truth_c_sqrtJ(
        theta_E=lens_param_samps[lens_idx,fp_idx,0],
        gamma_lens=lens_param_samps[lens_idx,fp_idx,3],
        R_sersic=R_sersic_truth,n_sersic=n_sersic_truth,
        beta_ani=beta_ani_samps[lens_idx,fp_idx])
    c_sqrtJ_samps[fp_idx,0] = csqrtJ

# filelock & write kinematics...
with FileLock(h5_posteriors_file + ".lock"):

    h5 = h5py.File(h5_posteriors_file, 'r+')
    h5['c_sqrtJ_samps'][lens_idx, ...] = c_sqrtJ_samps
    h5.close()

In [ ]:
# verify this works!
h5 = h5py.File(h5_posteriors_file, 'r')
print(h5['c_sqrtJ_samps'][0,:])
h5.close()